In [4]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [5]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
cd "/content/drive/My Drive/Colab Notebooks/DW_M/DW_M_ONE"

/content/drive/My Drive/Colab Notebooks/DW_M/DW_M_ONE


In [11]:
ls data

shoes_prices.csv  shoes_prices_small.csv


In [12]:
df = pd.read_csv('data/shoes_prices_small.csv', low_memory=False)
df.shape

(18375, 50)

In [0]:

def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)
   

In [14]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-60.490813740769, 5.359504389439481)

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-60.070643340558625, 5.415421844025242)

In [16]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-60.71415574444077, 5.304203394237332)

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-60.04511060445775, 5.56803609514033)

In [18]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat,manufacturer_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,-1
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,-1
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,-1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,-1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZS

In [19]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [20]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'


literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:

def parse_features(x):
   output_dict = {}
   if str(x) == 'nan': return output_dict
   
   features = literal_eval(x.replace('\\"', '"'))
   for item in features:
     key = item['key'].lower().strip()
     value = item['value'][0].lower().strip()
 
     output_dict[key] = value
 
   return output_dict
 
df['features_parsed'] = df['features'].map(parse_features)

In [22]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [23]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

478

In [24]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [25]:
df.columns


Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_eye protection type', 'feat_fabric material',
       'feat_lens color family', 'feat_gender',
       'feat_number of batteries included', 'feat_what includes',
       'feat_licensed', 'feat_shoe category', 'feat_shoe height',
       'feat_sock size'],
      dtype='object', length=529)

In [26]:
df[ df['feat_brand'].isnull() ].shape

(9481, 529)

In [27]:
df.shape[0]

18375

In [28]:
df[ False == df['feat_age start'].isnull() ].shape[0] / df.shape[0] * 100

0.4408163265306122

In [29]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0435374149659864

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [31]:
#keys_stat
{k:v for k,v in keys_stat.items() if v> 30}


{'brand': 48.40272108843537,
 'color': 47.57006802721088,
 'gender': 49.95918367346939,
 'manufacturer part number': 36.076190476190476,
 'material': 34.775510204081634}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_material_cat'] = df['feat_material'].map(lambda x: str(x).lower()).factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_style_cat'] = df['feat_style'].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

  

In [33]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
#df[df.brand == df.feat_brand][['brand', 'feat_brand']].shape
df[df.brand == df.feat_brand].head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_is recyclable_cat,feat_target audience_cat,feat_auto_cat,"feat_thick, warm, cozy_cat",feat_insulated_cat,feat_lens type_cat,feat_manufacturer number_cat,feat_band material_cat,feat_main color_cat,feat_display_cat,feat_outer material_cat,feat_style code_cat,feat_protects against_cat,feat_catalog_cat,feat_sleeve style_cat,feat_protection_cat,feat_fits brands_cat,feat_has paper wood_cat,feat_stylenumber_cat,feat_series_cat,feat_movement_cat,feat_reinforced pockets_cat,feat_shipping to usa_cat,feat_subdial_cat,feat_pocket_cat,feat_certifications and listings_cat,feat_combined shipping_cat,feat_looks best on these face shapes:_cat,feat_golf shoe type_cat,feat_metal stamp_cat,feat_product dimensions_cat,feat_eye protection type_cat,feat_fabric material_cat,feat_lens color family_cat,feat_number of batteries included_cat,feat_what includes_cat,feat_licensed_cat,feat_shoe category_cat,feat_shoe height_cat,feat_sock size_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1
1,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1
2,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen

In [34]:
#feats = ['']
df['feat_brand_cat'].head()

0    0
1    0
2    1
3    1
4    1
Name: feat_brand_cat, dtype: int64

In [35]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-60.000013049934296, 5.452516380157769)

In [36]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'manufacturer_cat',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_clothing category',
 'feat_recommended location',
 'feat_location - country',
 'feat_catalog',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_manufacturer_part_number_cat',
 'feat_sku number_cat',
 'feat_features_cat',
 'feat_stone type_cat',
 'feat_sports league_cat',
 'feat_handbag style_cat',
 'feat_rise_cat',
 'feat_batteries required?_cat',
 'feat_heat zones_cat',
 'feat_manufacturer sku_cat',
 'feat_number of wheels_cat',
 'feat_closure style_cat',
 'feat_country//region of manufacture_cat',
 'feat_assembled product weight_cat',
 'feat_hat style_cat',
 'feat_inseam_cat',
 'feat_boxed-product dimensions_cat',
 'feat_purpose_cat',
 'feat_fabric_cat',
 'feat_rx-able_cat',
 

In [0]:
feats = ['brand_cat','feat_metal type_cat','feat_shape_cat','feat_style_cat','feat_brand_cat','feat_ring size_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [38]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-60.452269420127024, 6.752932936494302)


Weight,Feature
0.3519 ± 0.0094,brand_cat
0.0426 ± 0.0043,feat_brand_cat
0.0413 ± 0.0046,feat_material_cat
0.0337 ± 0.0063,feat_gender_cat
0.0306 ± 0.0033,feat_shape_cat
0.0050 ± 0.0018,feat_style_cat
0.0043 ± 0.0007,feat_metal type_cat
0.0043 ± 0.0030,feat_style_cat
0.0016 ± 0.0003,feat_ring size_cat
0.0002 ± 0.0000,feat_sport_cat


In [39]:
df['brand'].value_counts(normalize=True)

nike                   0.096707
puma                   0.033143
ralph lauren           0.032816
vans                   0.021007
new balance            0.020190
                         ...   
saddle tramp           0.000054
nikon                  0.000054
gearonic tm            0.000054
rush by gordon rush    0.000054
adtec                  0.000054
Name: brand, Length: 1732, dtype: float64

In [40]:
#df[df['brand'] == 'nike'].features_parsed.head().values
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'shoe category': 'mens shoes', 'color': 'platinum/grey/anthracite', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '525762013', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'condition': 'new with box', 'type': 'cleats'},
       {'style': 'running, cross training', 'material': 'synthetic', 'country/region of manufacture': 'vietnam', 'condition': 'new with box'},
       {}, {}], dtype=object)

In [41]:
df['feat_age group'].value_counts()

adult               4565
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
mens                   4
toddler                4
boys'                  3
women ,�� unisex       2
youth                  2
men||women             2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [42]:
ls


data/  day3.ipynb  day4.ipynb  day5.ipynb


In [44]:
cd ..


/content/drive/My Drive/Colab Notebooks/DW_M


In [45]:
ls


DW_M_ONE/  hello_github.ipynb  LICENSE  README.md


In [0]:
!git add DW_M_ONE/day5.ipynb

In [55]:
!git commit -m "D5 Read MSP dataset from data.world."

[master d4e0571] D5 Read MSP dataset from data.world.
 1 file changed, 1 insertion(+)
 create mode 100644 DW_M_ONE/day5.ipynb


In [0]:
!git config --global user.email "lab@lubinski.ml"
!git config --global user.name "plubinski"

In [52]:
! git push -u origin master --force

fatal: could not read Password for 'https://a9ea4bd2a824ec570c77daaf2cf5b6715254746c@github.com': No such device or address


In [56]:
!git push origin master

fatal: could not read Password for 'https://a9ea4bd2a824ec570c77daaf2cf5b6715254746c@github.com': No such device or address
